In [1]:
%load_ext autoreload
%autoreload 2
import torch as t
import cv2 
import sys
sys.path.append("/data_nfs/je30bery/melanoma_data/model")
from model import EfficientnetWithFinetuning
from data import MelanomaData
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, f1_score, accuracy_score
from sklearn.decomposition import PCA
import random 
import pandas as pd
import seaborn as sns

plt.style.use('dark_background')
os.environ["CUDA_VISIBLE_DEVICES"]="1"

os.getcwd()

'/data_nfs/je30bery/melanoma_data/model/Clustering'

In [2]:
markers = ['ADAM10', 'Bcl-2', 'CD10', 'CD107a', 'CD13', 'CD138', 'CD14', 'CD1a', 'CD2', 'CD25', 'CD271', 'CD3', 'CD36', 'CD4', 'CD44', 'CD45', 'CD45RA', 'CD45RO', 'CD5', 'CD56', 'CD6', 'CD63', 'CD66abce', 'CD7', 'CD71', 'CD8', 'CD9', 'CD95', 'Collagen IV', 'Cytokeratin-14', 'EBF-P', 'EGFR', 'EGFR-AF488', 'HLA-ABC', 'HLA-DR', 'KIP1', 'Ki67', 'L302', 'MCSP', 'Melan-A', 'Nestin-AF488', 'Notch-1', 'Notch-3', 'PPARgamma', 'PPB', 'RIM3', 'TAP73', 'Vimentin', 'p63', 'phospho-Connexin']    

In [3]:
data = "/data_nfs/datasets/melc/melanoma/processed"
vdata = [os.path.join(data, "validation", d) for d in sorted(os.listdir(os.path.join(data, "validation")))] # balance_and_shuffle(os.path.join(data, "validation"))#
tdata = [os.path.join(data, "training", d) for d in sorted(os.listdir(os.path.join(data, "training")))] # balance_and_shuffle(os.path.join(data, "validation"))#
data = vdata + tdata
dl = t.utils.data.DataLoader(MelanomaData(markers, data, mode="val"), batch_size=10) #, shuffle=True)

FileNotFoundError: [Errno 2] No such file or directory: '/data_nfs/datasets/melc/melanoma/processed/validation'

In [ ]:
model = EfficientnetWithFinetuning(indim=len(markers))
model.load_state_dict(t.load("/data_nfs/je30bery/melanoma_data/model/saved_models/model_2023-11-20 16:27:56.558630_f1=0.9260606060606061_acc=0.9166666666666666_11.pt"))
model.eval()

In [ ]:
labels = np.array([int("Nevi" in d) for d in data])
labels

In [ ]:
def extract_features(model, x):
    model = model.cuda()
    x = x.cuda()
    x = model.features(x)
    x = model.avgpool(x)
    x = model.flatten(x)
    x = model.classifier1(x)    
    return x

In [ ]:
it = iter(dl)

In [ ]:
features = list()
while True:
    try:
        input_t, label = next(it)
    except StopIteration:
        break
    features.append(extract_features(model, input_t).detach().cpu().numpy())

In [ ]:
features = np.concatenate(features)
features.shape

In [ ]:
num_pertubations = 1000
shilouette_scores = list()
reference_scores = list()

mel_features = features[np.where(labels==0)]

for i in range(2,11):
    kmeans = KMeans(n_clusters=i, n_init="auto")
    y = kmeans.fit_predict(mel_features)
    y_copy = y.copy()
    score = silhouette_score(mel_features, y)
    shilouette_scores.append(score)

    acc = list()
    for j in range(num_pertubations):
        random.shuffle(y)
        acc.append(silhouette_score(mel_features, y))
    reference_scores.append(score - np.array(acc))

In [ ]:
gap_stat = pd.DataFrame(np.array(reference_scores).T, columns=range(2,11))

In [ ]:
plt.clf()
plt.title("K-Means Clustering of Melanoma Samples in 50-dim. Feature Space")
plt.xlabel("K")
plt.plot(range(0, 9), shilouette_scores, label="Silhouette score of\nK-Means clusters", color="m")
sns.pointplot(gap_stat, label="Gap: Difference of Sil. score\nof K-Means and clusters\nwith permuted labels, n=1000")
plt.legend(fontsize="8")
#plt.savefig("Gap_Statistics.pdf")

In [ ]:
plt.title("K-Means Clustering of Melanoma Samples in 50-dim. Feature Space \n Gap Statistics")
plt.ylabel("Gap")
plt.xlabel("K")
sns.pointplot(gap_stat)

In [ ]:
p = PCA(2)

In [ ]:
transf = p.fit_transform(features)
kmeans = KMeans(n_clusters=4, n_init="auto")
y = kmeans.fit_predict(features)

In [ ]:
f, axs = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(10, 5))
for c in np.unique(y):
    axs[0].scatter(transf[np.where(y == c)][:,0], transf[np.where(y == c)][:,1], label=f"Cluster {c}")
axs[0].legend()
plt.suptitle("PCA on Features")
axs[0].set_xlabel("Principal Component 1")
axs[0].set_ylabel("Principal Component 2")
axs[1].set_xlabel("Principal Component 1")


axs[1].scatter(transf[np.where(labels == 0)][:,0], transf[np.where(labels == 0)][:,1], label=f"Melanoma")
axs[1].scatter(transf[np.where(labels == 1)][:,0], transf[np.where(labels == 1)][:,1], label=f"Nevi")
axs[1].legend()

In [ ]:
f, axs = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(10, 5))
pal = sns.color_palette("Spectral", 42)
ids = np.array(["_".join(os.path.basename(id).split("_")[:2]) for id in data])
for i, c in enumerate(np.unique(ids)):
    if "Melanoma" in c:
        axs[0].scatter(transf[np.where(ids == c)][:,0], transf[np.where(ids == c)][:,1], label=c, color=pal[int(c.split("_")[-1])])
    else:
        axs[1].scatter(transf[np.where(ids == c)][:,0], transf[np.where(ids == c)][:,1], label=c, color=pal[int(c.split("_")[-1])*2])
plt.title("Coloring per Patient")
axs[0].set_title("Melanoma")
axs[1].set_title("Nevi")
plt.show()